# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096990


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:14,  2.67s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:04,  2.38s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:44,  1.70s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:31,  1.25s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:11<00:00,  4.57it/s]

Rendering models:  93%|█████████▎| 28/30 [00:11<00:00,  4.35it/s]

Rendering models:  97%|█████████▋| 29/30 [00:11<00:00,  4.93it/s]

Rendering models: 100%|██████████| 30/30 [00:11<00:00,  5.30it/s]

not-logged-in-3b0b929f4475fd276561    0.002485
ElisabethB                            0.000194
Jnursssmith5263                       0.001025
DevionJ                               0.002891
jnarayanbvg                           0.081161
Jonas_Cross                           0.000328
awright5                              0.001809
Borkdoge123                           0.000349
Person200                             0.000148
jaksonA                               0.444070
not-logged-in-f7e2260ed4a9517a3e4a    0.002185
not-logged-in-6c3c85931d83a1164a0d    0.003291
krolov                                0.000429
theetick                              0.000837
sn346808                              0.001074
Jakub21                               0.000478
Rmarks1701                            0.000942
mami2019                              0.002088
Nicola123                             0.001059
not-logged-in-eb3cb76b47f2a58893e0    0.000759
puic                                  0.000279
Tenorclef    

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())